In [ ]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

class FinalProject:
    
    def __init__(self, headless = True):

        from selenium import webdriver

        option = webdriver.ChromeOptions()
        option.add_argument('--lang=zh_TW-ZH_TW')   #繁體中文
        if headless:
            option.add_argument('--headless')       #隱藏頁面
        driver = webdriver.Chrome('./chromedriver', options=option)
        self.driver = driver    #設定好的driver
        self.set_dictionary()
        
    def set_dictionary(self):
        self.dictionary = {}
        websites = ["udn", "chinatimes", "tvbs", "nownews", "ftvnews", "apple", "ltn", "google", "yahoo", "youtube", "bing"]
        for website in websites:
            self.dictionary[website] = {}
        
    def wait_and_find(self, by, path):
        locator = (by, path)
        WebDriverWait(self.driver, 10, 0.5).until(EC.presence_of_element_located(locator))
        method = eval("self.driver.find_element_by_" + "_".join(str(by).split(".")[-1].lower().split()))
        return method(path)
    
    def wait_and_finds(self, by, path):
        locator = (by, path)
        WebDriverWait(self.driver, 10, 0.5).until(EC.presence_of_element_located(locator))
        method = eval("self.driver.find_elements_by_" + str(by).split(".")[-1].lower())
        return method(path)
    
    @staticmethod
    def print5(title, time, summary, link, picture):
        print("title: ",title)
        if time != None:
            print("time: ",time)
        if summary != None:
            print("summary:",summary)
        print("link: ",link)
        print("picture: ", picture)
        print()
        
    @staticmethod
    def imagepath():

        import os
        from tkinter import filedialog

        default_dir = r"C:\Users\Desktop"  # 設置默認打開目錄
        fname = filedialog.askopenfilename(title=u"選擇圖片",initialdir=(os.path.expanduser(default_dir)))

        return fname # 文件絕對路徑

    @staticmethod
    def path_is_image(path):

        import imghdr
        img = imghdr.what(path)   #檢查路徑是否為圖片

        if img != None:
            return True
        return False 
        
    def google_image(self):
        
        imagepath = FinalProject.imagepath()
        while  imagepath == "" or not (FinalProject.path_is_image(imagepath)) :
            print("請選擇一張圖片!!")
            imagepath = FinalProject.imagepath()

        #打開google圖片
        self.driver.get('https://www.google.com.tw/imghp')
        imagebutton = self.wait_and_find(By.CLASS_NAME, "LM8x9c")
        imagebutton.click()

        #傳送圖片
        image = self.wait_and_find(By.NAME, "encoded_image")
        image.send_keys(imagepath)

        #取得搜尋結果
        q = self.wait_and_find(By.NAME, "q")
        image_response = q.get_attribute("value")

        return image_response

    def google_translate(self, image_response):

        #打開google翻譯並輸入文字
        self.driver.get('https://translate.google.com/')
        transinput = self.wait_and_find(By.ID, "source")
        transinput.send_keys(image_response)

        #取得中文翻譯以及原文語言
        translate_response = self.wait_and_find(By.XPATH, """/html/body/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/div/span[1]""").text        
        lang = self.driver.find_element_by_xpath("""/html/body/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]""").text.split()[0]

        print()
        print("中文: " + translate_response)
        print()

        #取得英文翻譯
        if lang == "英文":
            print("英文: " + image_response)
        else:
            englishbutton = self.driver.find_elements_by_id("sugg-item-en")[1]
            englishbutton.click()    #點擊英文翻譯
            english = self.wait_and_find(By.XPATH, """/html/body/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[3]/div[1]/div[2]/div/span[1]/span""").text
            print("英文: " + english)

            if lang != "中文":
                #原文非中文,英文
                print(lang + ": " + image_response)
        print()

        return translate_response

    def wikipedia(self, translate_response):
        #查詢維基百科
        self.driver.get("https://www.google.com.tw/")
        q = self.wait_and_find(By.NAME, "q")
        q.send_keys(translate_response+" 維基百科")
        q.send_keys(Keys.RETURN)

        #點擊第一項名字有維基百科的搜尋結果
        self.wait_and_find(By.CLASS_NAME, "q")
        g = self.driver.find_elements_by_class_name("LC20lb")
        for title in g:
            if "維基百科" in title.text:
                title.click()
                break

        #找尋解釋文字
        wikitext = self.wait_and_find(By.XPATH,"""//*[@id="mw-content-text"]/div/p""").text
        print(wikitext)

        try:      
            disambiguation = self.wait_and_find(By.CLASS_NAME,"mbox-text")

            if "消歧義" in disambiguation.text or "消歧义" in disambiguation.text:
                #處理消歧義頁面問題 
                alldisambiguation = self.driver.find_elements_by_xpath("""//*[@id="mw-content-text"]/div/ul/li/a[1]""")            
                l = len(alldisambiguation)   #取得子頁面數量

                for i in range(l):
                    print()
                    disambiguation_i = self.wait_and_find("""//*[@id="mw-content-text"]/div/ul/li/a[1]""")[i]
                    disambiguation_i_text = self.driver.find_elements_by_xpath("""//*[@id="mw-content-text"]/div/ul/li""")[i].text
                    print(disambiguation_i_text)   #子頁面名稱
                    disambiguation_i.click()

                    #取得子頁面解釋
                    subtext = self.self.wait_and_find(By.XPATH,"""//*[@id="mw-content-text"]/div/p""").text
                    print(subtext)
                    self.driver.back()
        except:
            #無消歧義
            pass
        print()

    def cezisuanming(self, translate_response):

        #打開諸葛神數
        self.driver.get('https://www.ximizi.net/zhuge_shenshu.php')
        poeminput = self.wait_and_find(By.NAME,"cezisuanming")
        poeminput.send_keys(translate_response)     #輸入中文
        poeminput.send_keys(Keys.RETURN)

        #取得籤詩及其解釋
        poem = self.wait_and_find(By.XPATH,"""/html/body/div[1]/div[6]/div[3]/p[2]/font""").text  
        poem_analysis = self.driver.find_element_by_xpath("""/html/body/div[1]/div[6]/div[3]/p[3]""").text

        print("籤詩: " + poem)
        print()
        print(poem_analysis)
        print()
    
    def udn(self, search):
        if search in self.dictionary["udn"].keys():
            for title, time, summary, link, picture in self.dictionary["udn"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://udn.com/search/result/2/" + search
            self.driver.get(html)
            self.dictionary["udn"][search] = []
            soup = BeautifulSoup(self.driver.page_source, "html.parser")
            search_content = soup.find("div", id = "search_content").find_all("a")
            for i in search_content:
                link = i.get("href")
                title = i.find("h2").text
                time = i.find("span").text.split("：")[-1]
                summary = i.find("p").text
                picture = i.find("img").get("src")
                self.dictionary["udn"][search].append((title, time, summary, link, picture))
                FinalProject.print5(title, time, summary, link, picture)
                
    def chinatimes(self, search):
        if search in self.dictionary["chinatimes"].keys():
            for title, time, summary, link, picture in self.dictionary["chinatimes"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://www.chinatimes.com/search/" + search + "?chdtv"
            self.driver.get(html)
            self.dictionary["chinatimes"][search] = []
            soup = BeautifulSoup(self.driver.page_source, "html.parser")
            search_content = soup.find("ul", class_ = "vertical-list list-style-none").find_all("li")
            for i in search_content:
                if i.get("id") == None:
                    h3 = i.find("h3")
                    title = h3.text
                    a = h3.find("a")
                    link = a.get("href")
                    time_list = i.find("time").find_all("span")
                    time = time_list[0].text + " " + time_list[1].text
                    summary = i.find("p").text
                    picture = i.find("img").get("src")
                    self.dictionary["chinatimes"][search].append((title, time, summary, link, picture))
                    FinalProject.print5(title, time, summary, link, picture)
                    
    def tvbs(self, search):
        if search in self.dictionary["tvbs"].keys():
            for title, time, summary, link, picture in self.dictionary["tvbs"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://news.tvbs.com.tw/news/searchresult/news?search_text=" + search
            self.driver.get(html)        
            self.dictionary["tvbs"][search] = []
            soup = BeautifulSoup(self.driver.page_source, "html.parser")
            search_content = soup.find("div", class_ = "search_list_div").find_all("li")
            for i in search_content:
                a = i.find("a")
                link = a.get("href")
                title = a.find("div", class_ = "search_list_txt").text
                time = a.find("div", class_ = "icon_time").text
                picture = i.find("img").get("src")
                self.dictionary["tvbs"][search].append((title, time, None, link, picture))
                FinalProject.print5(title, time, None, link, picture)
    
    def nownews(self, search):
        if search in self.dictionary["nownews"].keys():
            for title, time, summary, link, picture in self.dictionary["nownews"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://www.nownews.com/contentsearch/?q=" + search
            self.driver.get(html)
            self.dictionary["nownews"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find_all("div", class_ = "gsc-webResult gsc-result")
            for i in search_content:
                gs_title = i.find("a", class_ = "gs-title")
                title, link= gs_title.text, gs_title.get("href")
                temp = i.find("div", class_ = "gs-bidi-start-align gs-snippet").text.split("...")
                time, summary = temp[0], temp[1]
                picture = i.find("img").get("src")
                self.dictionary["nownews"][search].append((title, time, summary, link, picture))
                FinalProject.print5(title, time, summary, link, picture)

    def ftvnews(self, search):
        if search in self.dictionary["ftvnews"].keys():
            for title, time, summary, link, picture in self.dictionary["ftvnews"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://www.ftvnews.com.tw/search?key=" + search
            self.driver.get(html)
            self.dictionary["ftvnews"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find("section", class_ = "search-list clearfix").find_all("li")
            for i in search_content:
                link = "https://www.ftvnews.com.tw/" + i.find("a").get("href")
                time = " ".join(i.find("span", class_ = "time").text.split())
                title = i.find("div", class_ = "title").text
                summary = i.find("div", class_ = "summary").text
                picture = i.find("img").get("src")
                self.dictionary["ftvnews"][search].append((title, time, summary, link, picture))
                FinalProject.print5(title, time, summary, link, picture)
                
    def apple(self, search):
        if search in self.dictionary["apple"].keys():
            for title, time, summary, link, picture in self.dictionary["apple"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://tw.appledaily.com/search/result?querystrS=" + search
            self.driver.get(html)
            self.dictionary["apple"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find("ol", id = "result").find_all("div", class_ = "content")    
            for i in search_content:
                a = i.find("a")
                title, link = " ".join(a.text.split()), a.get("href")
                summary = i.find("p", class_ = "ellipsis").text
                time = i.find("time").text
                self.dictionary["apple"][search].append((title, time, summary, link, None))
                FinalProject.print5(title, time, summary, link, None)
                
    def ltn(self, search):
        if search in self.dictionary["ltn"].keys():
            for title, time, summary, link, picture in self.dictionary["ltn"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://news.ltn.com.tw/search?keyword=" + search
            self.driver.get(html)
            self.dictionary["ltn"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find("ul", class_ = "searchlist boxTitle").find_all("li")
            for i in search_content:
                time = i.find("span").text
                a = i.find("a", class_ = "tit")
                title, link = a.text, a.get("href")
                summary = "".join(i.find("p").text.split())
                picture = i.find("img").get("src")
                self.dictionary["ltn"][search].append((title, time, summary, link, None))
                FinalProject.print5(title, time, summary, link, None)
                
    def google(self, search):
        if search in self.dictionary["google"].keys():
            for title, time, summary, link, picture in self.dictionary["google"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://www.google.com/search?q=" + search
            self.driver.get(html)
            self.dictionary["google"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find_all("div", class_ = "g")
            for i in search_content:
                try:
                    h3 = i.find("h3", class_ = "LC20lb")
                    title = h3.text
                    a = h3.find_parent("a")
                    link = a.get("href")
                    summary = i.find("span", class_ = "st").text
                    self.dictionary["google"][search].append((title, None, summary, link, None))
                    FinalProject.print5(title, None, summary, link, None)
                except:
                    pass
            
    def yahoo(self, search):
        if search in self.dictionary["yahoo"].keys():
            for title, time, summary, link, picture in self.dictionary["yahoo"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://tw.search.yahoo.com/search?p=" + search
            self.driver.get(html)
            self.dictionary["yahoo"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find("div", id = "web").find_all("li", class_ = None)
            for i in search_content:
                try:
                    h3 = i.find("h3", class_ = "title")
                    title = h3.text
                    link = h3.find("a").get("href")
                    summary = i.find("div", class_ = "compText aAbs").text
                    self.dictionary["yahoo"][search].append((title, None, summary, link, None))
                    FinalProject.print5(title, None, summary, link, None)
                except:
                    pass
                
    def youtube(self, search):
        if search in self.dictionary["youtube"].keys():
            for title, time, summary, link, picture in self.dictionary["youtube"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://www.youtube.com/results?search_query=" + search
            self.driver.get(html)
            self.dictionary["youtube"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find("div", class_ = "style-scope ytd-vertical-list-renderer").find_all("ytd-video-renderer")
            #print(search_content)
            for i in search_content:
                a = i.find("a", id = "thumbnail")
                link = "https://www.youtube.com" + a.get("href")
                picture = a.find("img").get("src")
                title = "".join(i.find("div", id = "title-wrapper").find("h3").text.split())
                time = " ".join(i.find("div", id = "metadata").text.split())
                summary = "".join(i.find("yt-formatted-string", id = "description-text").text.split())
                self.dictionary["youtube"][search].append((title, time, summary, link, picture))
                FinalProject.print5(title, time, summary, link, picture)
                
    def bing(self, search):
        if search in self.dictionary["bing"].keys():
            for title, time, summary, link, picture in self.dictionary["bing"][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = "https://www.bing.com/search?q=" + search
            self.driver.get(html)
            self.dictionary["bing"][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = soup.find("ol", id = "b_results").find_all("li")
            for i in search_content:
                try:
                    a = i.find("a")
                    title, link = a.text, a.get("href")
                    summary = i.find("div", class_ = "b_caption").find("p").text
                    self.dictionary["bing"][search].append((title, None, summary, link, None))
                    FinalProject.print5(title, None, summary, link, None)
                except:
                    pass
        

In [ ]:

driver = FinalProject()
driver.nownews("韓國瑜")

In [ ]:
print("self.driver.find_element_by_" + str(By.CLASS_NAME).split(".")[-1].lower())
By.CLASS_NAME.split(".")[-1]

In [ ]:
driver.bing("韓國瑜")

In [ ]:
def (self, search):
        if search in self.dictionary[""].keys():
            for title, time, summary, link, picture in self.dictionary[""][search]:
                FinalProject.print5(title, time, summary, link, picture)
        else:
            html = 
            self.driver.get(html)
            self.dictionary[""][search] = []
            soup = BeautifulSoup(driver.driver.page_source, "html.parser")
            search_content = 
            for i in search_content:                
                self.dictionary[""][search].append((title, time, summary, link, picture))
                FinalProject.print5(title, time, summary, link, picture)